# Generate Jobs Report Data for Chartbook

Brian Dew

@bd_econ

In [2]:
import sys
import json
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Unemployment rate

In [2]:
# Series stored as a dictionary
series = {'LNS14000003': 'White', 
          'LNS14000006': 'Black', 
          'LNS14000009': 'Hispanic',
          'LNS14000000': 'Total',
          'LNS13000000': 'Level'}

# Start year and end year
dates = (1989, 2019)
df = bls_api(series, dates, bls_key)
srs = ['White', 'Black', 'Hispanic']
df[srs].to_csv(data_dir / 'unemp.csv', index_label='date')

s = series_info(df['Level'])
s2 = series_info(df['Total'])
s3 = series_info(df['Black'])
text = ('Unemployment is currently very low. BLS '+
        '\href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} '+
        f'{s["val_latest"]/1000:.1f} million '+
        f'unemployed persons in {s["date_latest_ft"]}, '+
        f'and an unemployment rate of {s2["val_latest"]} percent. '+
        'However, unemployment is much higher for disadvantaged groups, '+
        'with the black or African American unemployment rate typically double '+
        'the white unemployment rate. '+
        'A very tight labor market may have the effect of reducing racial '+
        'discrimination in hiring. '+
        'Over the past year, the black or African American unemployment rate '+
        f'has fallen by {abs(s3["change_year_ago"]):.1f} percentage '+
        f'points to {s3["val_latest"]:.1f} percent.')
write_txt(text_dir / 'unemp.txt', text)
text

Post Request Status: REQUEST_SUCCEEDED


'Unemployment is currently very low. BLS \\href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} 5.8 million unemployed persons in December 2019, and an unemployment rate of 3.5 percent. However, unemployment is much higher for disadvantaged groups, with the black or African American unemployment rate typically double the white unemployment rate. A very tight labor market may have the effect of reducing racial discrimination in hiring. Over the past year, the black or African American unemployment rate has fallen by 0.7 percentage points to 5.9 percent.'

### Employment rate

In [8]:
series = {'LNS12300060': 'PA_EPOP'}

# Start year and end year
dates = (1989, 2019)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'epop.csv', index_label='date')

s = pd.read_csv(data_dir / 'epop.csv').iloc[-1]
date = pd.to_datetime(s.date).strftime('%Y-%m')
value = s.PA_EPOP

txt = f'{date}:\\\{value}\%'

f = text_dir.joinpath('epop.txt')
with f.open('w') as wf:
    wf.write(txt)
    
s = pd.read_csv(data_dir / 'epop.csv', parse_dates=['date']).set_index('date')['PA_EPOP']
d = series_info(s)

text = f'In {d["date_latest_ft"]}, {d["val_latest"]}\%'

if (d['days_since_match'] > 725) | (d['days_since_match'] == 0):
    text2 = d['last_matched']
else:
    text2 = f'compared to {d["val_prev"]} in {d["date_prev_ft"]}'
    
if d['change_year_ago'] > 0:
    direction = 'increased'
    value = d['change_year_ago']
    label = f'Over the past year, the age 25-54 employment rate has {direction} by {value:.1f} percentage points.'
elif d['change_year_ago'] < 0:
    direction = 'fallen'
    value = abs(d['change_year_ago'])
    label = f'Over the past year, the age 25-54 employment rate has {direction} by {value:.1f} percentage points.'
else:
    label = 'The age 25-54 employment rate is unchanged over the past year. '
    
pop = 126277
diff1 = d['late90s'] - d['val_latest']
diff = (d['late90s'] - d['val_latest']) / 100 * pop

if diff > 999:
    diff_text = f'{round(diff / 1000, 1)} million'
else: 
    diff_text = f'{round(diff, -1)} thousand'
    
label2 = f'The current age 25-54 employment rate is {diff1:.1f} percentage points (equivalent to {diff_text} workers) below the average during 1998--99, a period with a particularly tight labor market. '

textval = f'{text} of 25-54 years olds were employed, {text2}. {label} {label2}'
print(textval)

f = text_dir.joinpath('epop_text.txt')
with f.open('w') as wf:
    wf.write(textval)

In December 2019, 80.4\% of 25-54 years olds were employed, the highest level since July 2001. Over the past year, the age 25-54 employment rate has increased by 0.8 percentage points. The current age 25-54 employment rate is 0.9 percentage points (equivalent to 1.1 million workers) below the average during 1998--99, a period with a particularly tight labor market. 


### Unemployment by reason

In [5]:
# Series stored as a dictionary
series = {'LNS14023621': 'Job Loser', 
          'LNS14023705': 'Job Leaver', 
          'LNS14023557': 'Re-entrant',
          'LNS14023569': 'New entrant'}

# Start year and end year
dates = (1989, 2019)
df = bls_api(series, dates, bls_key)

df.resample('QS').mean().to_csv(data_dir / 'unemp_reason.csv', index_label='date', float_format='%g')

loser = df['Job Loser'].iloc[-1]
leaver = df['Job Leaver'].iloc[-1]
reent = df['Re-entrant'].iloc[-1]
newent = df['New entrant'].iloc[-1]
ltdate = df.index[-1].strftime('%B %Y')

text = (f'In {ltdate}, {loser:.1f} percent of the labor force '+
        'were unemployed because of losing a job or having a '+
        f'job end, {leaver:.1f} percent were re-entrants, '+
        f'{reent:.1f} percent new entrants, and {newent:.1f} '+
        'percent job leavers. ')

write_txt(text_dir / 'unemp_reason.txt', text)

Post Request Status: REQUEST_SUCCEEDED


### Flows

In [6]:
# Series stored as a dictionary
series = {'LNS17200000': 'NILF',
          'LNS17100000': 'UNEMP'}

# Start year and end year
dates = (1990, 2019)
data = bls_api(series, dates, bls_key)

df = data
df['TOTAL'] = data.astype('float').sum(axis=1)
sh = (df['NILF'] / df['TOTAL']).rename('total') * 100

sh.to_csv(data_dir / 'lf_flow.csv', index_label='date', header=True, float_format='%g')

sh.resample('QS').mean().rename('quarterly').to_csv(data_dir / 'lf_flow_q.csv', index_label='date', header=True)

totval = df['TOTAL'].iloc[-1] / 1000
nilfval = df['NILF'].iloc[-1] / 1000
unval = df['UNEMP'].iloc[-1] / 1000

shval = sh.iloc[-1]

sh3y = sh.iloc[-37]

ltdate = sh.index[-1].strftime('%B %Y')
yragodt = sh.index[-37].strftime('%B %Y')

text = (f'In {ltdate}, {totval:.1f} million people were newly employed (on a gross basis). '+
        f'Of these, {shval:.1f} percent were not looking for work in the prior '+
        'month. With low unemployment, new employees are being pulled '+
        'from outside of the labor force and bypassing unemployment. '+
        f'Three years ago, in {yragodt}, {sh3y:.1f} percent '+
        'of the newly employed were not looking for work month prior.')

write_txt(text_dir / 'lf_flow.txt', text)

Post Request Status: REQUEST_SUCCEEDED


### Wage Growth

In [7]:
data1, data2 = pd.Series(), pd.Series()
columns = ['MONTH', 'YEAR', 'AGE', 'PWORWGT', 'WKWAGE', 'HRSUSL1', 'WORKFT']
for year in range(1989, 2020):
    df = (pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
        .query('WKWAGE > 0 and WORKFT == 1'))
    data = df.groupby(['YEAR', 'MONTH']).apply(binned_wage)
    data.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data.index]
    data1 = data1.append(data)
    
df = pd.DataFrame({'All': data1})

df.rolling(3).mean().to_csv(data_dir / 'uwe_bd.csv', index_label='date')
(df.pct_change(12).dropna() * 100).rolling(3).mean().to_csv(data_dir / 'uwe_bd_gr.csv', index_label='date')

# Series stored as a dictionary
series = {'LEU0252911200': 'value'}

# Start year and end year
dates = (2000, 2019)
df2 = bls_api(series, dates, bls_key)

df2.to_csv(data_dir / 'uwe_bls.csv', index_label='date')
(df2.pct_change(4).dropna() * 100).to_csv(data_dir / 'uwe_bls_gr.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### Average Hourly Earnings

In [3]:
series = {'CES0500000003': 'ALL', 'CES0500000008': 'PNS'}
years = (1988, 2019)
df = bls_api(series, years, bls_key)
(df.pct_change(12) * 100).loc['1989':].to_csv(data_dir / 'ahe.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [6]:
series = {'CES3000000008': 'Manufacturing',
          'CES1000000008': 'Mining \& Logging',
          'CES4422000008': 'Utilities',
          'CES4142000008': 'Wholesale Trade',
          'CES5000000008': 'Information',
          'CES5500000008': 'Financial Activities',
          'CES6000000008': 'Professional \& Business Services',
          'CES6500000008': 'Education \& Health Services',
          'CES0500000008': 'Total Private',
          'CES2000000008': 'Construction',
          'CES7000000008': 'Leisure \& Hospitality',
          'CES4300000008': 'Transportation \& Warehousing',
          'CES4200000008': 'Retail Trade'}

years = (2017, 2019)
df = bls_api(series, years, bls_key)

KeyError: 'series'

In [7]:
df

,PNS,ALL
1988-01-01,9.29,NaN
1988-02-01,9.29,NaN
1988-03-01,9.31,NaN
1988-04-01,9.36,NaN
1988-05-01,9.41,NaN
...,...,...
2019-08-01,23.60,28.11
2019-09-01,23.67,28.12
2019-10-01,23.73,28.20
2019-11-01,23.77,28.29


In [ ]:
((df.pct_change(12).iloc[-1] * 100.0)
    .sort_values(ascending=False)
    .to_csv(data_dir / 'ahe_ind.csv', index_label='name', header=True))

write_txt(text_dir / 'ahe_bar_date.txt', df.index[-1].strftime('%B %Y'))

text = ('\\noindent \hspace*{-2mm} \\begin{tikzpicture}'+
        '\\begin{axis}[\\barplotnogrid axis y line=left, \\barylab{4.0cm}{1.5ex}'+
        'width=5.2cm, bar width=1.8ex, height=7.2cm, xtick={0}, xmajorgrids,'+
        'enlarge y limits={abs=3mm}, enlarge x limits=0.1, '+
        f'\dbar{{x}}{{{allitems:.2f}}}, clip=false,'+
        'yticklabels from table={\\ahe}{name},'+
        'yticklabel style={font=\\footnotesize},'+
        'nodes near coords style={/pgf/number format/.cd, fixed zerofill,'+
        'precision=1, assume math mode}]'+
        '\\addplot[fill=blue!80!black, draw=none] '+
        'table [y expr=-\coordindex, x index=1] {\\ahe};'+
        f'\\node[right] at ({allitems:.2f}, -12.6) {{\\footnotesize \\textcolor{{black!80}}{{CPI}}}};'+
        '\end{axis}'+
        '\end{tikzpicture}\\\ '+
        '\\footnotesize{Source: Bureau of Labor Statistics} \hspace{32mm} \\tbllink{ahe_ind.csv}')

write_txt(text_dir / 'ahe_chart.txt', text)

print(text)